# Quiz 2

In [52]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## Problem 1

1. Generate $x$ by a uniform distribution in $[-1,1]$
2. Generate $y$ by $f(x)=\tilde{s}(x)$ where $\tilde{s}(x)=sign(x)$ and the noise flips the result with $20\%$ probability.

In [47]:
def run_stump(n):
    x = 1-2*np.random.random(n)
    y = np.sign(x)
    for i in range(len(y)):
        if np.random.random() < 0.2:
            y[i] = -y[i]
    
    x_sorted = [-1, *sorted(x)]
    thetas = [(a+b)/2 for a, b in zip(x_sorted[:-1], x_sorted[1:])]
    
    res = []
    min_Ein = 100*n
    for s in [-1, 1]:
        for theta in thetas:
            Ein = (1/n)*np.sum(s*np.sign(x-theta) != y)
            if Ein < min_Ein:
                res = [(s, theta)]
                min_Ein = Ein
            elif Ein == min_Ein:
                res.append((s, theta))
    
    s, theta = res[np.random.randint(0, len(res))]
#     print(s, theta)
    Eout = 0.5 + 0.3*s*(np.abs(theta)-1)
    return (min_Ein, Eout)

In [50]:
Ein_s, Eout_s = 0, 0
for rep in range(5000):
    Ein, Eout = run_stump(20)
    Ein_s, Eout_s = Ein_s + Ein, Eout_s + Eout
print('Average Ein:', Ein_s / 5000)
print('Average Eout:', Eout_s / 5000)

Average Ein: 0.16981999999999997
Average Eout: 0.2544467113781448


## Problem 2

In [65]:
def read_data(filename):
    dat = pd.read_csv(filename, sep=' ', header=None).to_numpy()[:, 1:]
    return dat[:,:-1], dat[:,-1]

In [108]:
def nd_stumps(X, y):
    n, D = X.shape
    
    outer_res = []
    outer_min_Ein = 1e10
    for d in range(D):
        x_sorted = [-1, *sorted(X[:,d])]
        thetas = [(2*a+3*b)/5 for a, b in zip(x_sorted[:-1], x_sorted[1:])]

        res = []
        min_Ein = 100*n
        for s in [-1, 1]:
            for theta in thetas:
                Ein = (1/n)*np.sum(s*np.sign(X[:,d]-theta) != y)
                if Ein < min_Ein:
                    res = [(s, theta, d)]
                    min_Ein = Ein
                elif Ein == min_Ein:
                    res.append((s, theta, d))

#         s, theta = res[np.random.randint(0, len(res))]
        if min_Ein < outer_min_Ein:
            outer_res = [r for r in res]
            outer_min_Ein = min_Ein
        elif min_Ein == outer_min_Ein:
            outer_res.extend(res)
    
    s, theta, d = outer_res[np.random.randint(0, len(outer_res))]
    print(outer_min_Ein, s, theta, d)
    return s, theta, d

In [109]:
X_train, y_train = read_data('hw2_train.dat')

In [110]:
s, theta, d = nd_stumps(X_train, y_train)

0.25 -1 1.6488 3


In [111]:
X_test, y_test = read_data('hw2_test.dat')

In [112]:
(1/len(y_test))*np.sum(s*np.sign(X_test[:,d]-theta) != y_test)

0.358